In [12]:
from __future__ import unicode_literals
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import
import argparse
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from dltk.networks.regression_classification.resnet import resnet_3d
from dltk.networks.segmentation.unet import residual_unet_3d
from dltk.core.activations import leaky_relu
from dltk.io.abstract_reader import Reader
import json

import SimpleITK as sitk
from dltk.io.augmentation import flip
from dltk.io.preprocessing import whitening
from dltk.io.preprocessing import resize_image_with_crop_or_pad
# from dltk.io.augmentation import extract_class_balanced_example_array
from patch import extract_class_balanced_example_array

from tensorflow.contrib import predictor
from dltk.io.augmentation import extract_random_example_array
import time

# ReaderX

In [13]:
def read_fn(file_references, mode, params=None):
    """A custom python read function for interfacing with nii image files.
    Args:
        file_references (list):
        mode (str): One of the tf.estimator.ModeKeys strings: TRAIN, EVAL or
            PREDICT.
        params (dict, optional): A dictionary to parametrise read_fn outputs
            (e.g. reader_params = {'n_patches': 10, 'patch_size':
            [64, 64, 64], 'extract_patches': True}, etc.).
    Yields:
        dict: A dictionary of reader outputs for dltk.io.abstract_reader.
    """

    def _augment(img):
        return flip(img, axis=2)

    for f in file_references:
        
        # Column 1: Image
        img_fn     = str(f[0])
        subject_id = img_fn.split('/')[-1].split('.')[0]
        img        = sitk.ReadImage(img_fn, sitk.sitkFloat32)       # Loading NIFTI Images via SimpleITK
        img        = sitk.GetArrayFromImage(img)
        img        = whitening(img)                                 # Whitening Transformation (Variance=1)
        
        # Column 2: Binary Mask
        binary_mask_fn  = str(f[1])
        binary_mask     = sitk.GetArrayFromImage(sitk.ReadImage(binary_mask_fn)).astype(np.int32)
        
        # Column 3: Probabilities Mask
        probs_mask_fn   = str(f[2])
        probs_mask      = sitk.GetArrayFromImage(sitk.ReadImage(probs_mask_fn, sitk.sitkFloat32))
        probs_mask      = np.flip(probs_mask,axis=1)                                                 # ++: Future Optimization: Preprocess in Prior (Flip)
        
        # Column 4: Feature Maps                                                                     # ++: Future Optimization: Preprocess in Prior (5D->4D)
        feature_maps_fn = str(f[3])
        # Loading 5D NIFTI Images as 4D Image
        file_reader = sitk.ImageFileReader()
        file_reader = sitk.ImageFileReader()
        file_reader.SetFileName(feature_maps_fn)
        file_reader.ReadImageInformation()
        feature_maps_size = list(file_reader.GetSize())
        file_reader.SetExtractSize([0 if v == 1 else v for v in feature_maps_size])
        feature_maps = file_reader.Execute()
        feature_maps = sitk.Compose( [sitk.Extract(feature_maps, feature_maps.GetSize()[:3]+(0,), [0,0,0, i]) for i in range(feature_maps_size[-1])] )
        feature_maps = sitk.GetArrayFromImage(feature_maps)
        
        



        # Padding Images
        patch_size = params['patch_size']
        img_shape  = img.shape
        
        # Padding Images --Z Dimension
        if (patch_size[0] >=img_shape[0]):
             zdim = patch_size[0]+10
        else:
             zdim = img_shape[0]
        # Padding Images --X Dimension
        if (patch_size[1] >=img_shape[1]):
             xdim = patch_size[1]+10
        else:
             xdim = img_shape[1]
        # Padding Images --Y Dimension
        if (patch_size[2] >=img_shape[2]):
             ydim = patch_size[2]+10
        else:
             ydim = img_shape[2]
        
        img            = resize_image_with_crop_or_pad(img,          [zdim,xdim,ydim],                         mode='symmetric')
        binary_mask    = resize_image_with_crop_or_pad(binary_mask,  [zdim,xdim,ydim],                         mode='symmetric')
        probs_mask     = resize_image_with_crop_or_pad(probs_mask,   [zdim,xdim,ydim],                         mode='symmetric')
        feature_maps    = resize_image_with_crop_or_pad(feature_maps,  [zdim,xdim,ydim,feature_maps.shape[3]], mode='symmetric')
        
        


        # Column 5: Label
        lbl         = np.int(f[4])
        lbl         = np.expand_dims(lbl, axis=-1).astype(np.int32)
        
        # Expand to 4D
        img         = np.expand_dims(img, axis=3)
        probs_mask  = np.expand_dims(probs_mask, axis=3)



        # Probabilities + Feature Map Concatenation
        connect = 'max'
        if connect=='probs_mask':
            img = np.concatenate((img,probs_mask),axis=3)
        elif connect=='feature_maps':    
            img = np.concatenate((img,feature_maps),axis=3)
        elif connect=='max':
            img = np.concatenate((img,probs_mask,feature_maps),axis=3)




        # TensorFlow Mode-Based Execution
        # Prediction Mode
        if mode == tf.estimator.ModeKeys.PREDICT:
            yield {'features': {'x': img},
                   'labels':   {'y': lbl.astype(np.float32)}, 
                   'img_id':         subject_id}
        
        # Training Mode
        if mode == tf.estimator.ModeKeys.TRAIN:
            pass
        
        # Augmentation Flag    
        if params['augmentation']:
            img = _augment(img)

        # Return Training Examples
        if params['extract_patches']:
            img,binary_mask = extract_class_balanced_example_array(
                img,binary_mask,
                example_size  = params['patch_size'],
                n_examples    = params['n_patches'],
                classes       = 4, class_weights=[0,0,1,1]   # Label 3,4 => Right,Left Lungs (XCAT)
                )

            for e in range(params['n_patches']):
                yield {'features': {'x': img[e].astype(np.float32)},
                       'labels':   {'y': lbl.astype(np.float32)},
                       'img_id':         subject_id}

        # Return Full Images
        else:
            yield {'features': {'x': img},
                   'labels':   {'y': lbl.astype(np.float32)},
                   'img_id':         subject_id}

    return

# ModelX

In [14]:
def lrelu(x):
    return leaky_relu(x, 0.1)

def model_fn(features, labels, mode, params):

    # Segmentation Model Definition (3D Residual U-Net)
    segnet_output_ops = residual_unet_3d(
        inputs              = features['x'],                               # Input: Patches (dimensions=128cube; defined by 'patch_size')
        num_classes         = 2,
        num_res_units       = 3,
        filters             = [16, 64, 128, 256, 512],
        strides             = ((1, 1, 1), (2, 2, 2), (2, 2, 2), (2, 2, 2), (1, 1, 1)),
        mode                = tf.estimator.ModeKeys.EVAL,
        activation          = lrelu,
        kernel_initializer  = tf.initializers.variance_scaling(distribution='uniform'),
        bias_initializer    = tf.zeros_initializer(),
        kernel_regularizer  = None,
        bias_regularizer    = None)



    # Classification Model Definition (3D ResNet)
    clfnet_output_ops = resnet_3d(
        inputs              = features['x'],                # Input: Output of 3D Residual U-Net (dimensions=128cube)
        num_res_units       = 2,
        num_classes         = NUM_CLASSES,
        filters             = (16, 32, 64, 128, 256),
        strides             = ((1, 1, 1), (2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2)),
        kernel_initializer  = tf.initializers.variance_scaling(distribution='uniform'),
        bias_initializer    = tf.zeros_initializer(),
        mode                = mode,
        kernel_regularizer  = tf.contrib.layers.l2_regularizer(1e-3))

    # Prediction Mode
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode            = mode,
            predictions     = clfnet_output_ops,
            export_outputs  = {'out': tf.estimator.export.PredictOutput(clfnet_output_ops)})

    # Loss Function
    one_hot_labels = tf.reshape(tf.one_hot(labels['y'], depth=NUM_CLASSES), [-1, NUM_CLASSES])
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels      = one_hot_labels,
        logits             = clfnet_output_ops['logits'])

    # Optimizer
    global_step = tf.train.get_global_step()
    if params["opt"] == 'adam':
        optimiser = tf.train.AdamOptimizer(
            learning_rate=params["learning_rate"], epsilon=1e-5)
    elif params["opt"] == 'momentum':
        optimiser = tf.train.MomentumOptimizer(
            learning_rate=params["learning_rate"], momentum=0.9)
    elif params["opt"] == 'rmsprop':
        optimiser = tf.train.RMSPropOptimizer(
            learning_rate=params["learning_rate"], momentum=0.9)

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimiser.minimize(loss, global_step=global_step)

    # Custom Image Summaries (TensorBoard)
    my_image_summaries = {}
    my_image_summaries['CT_Patch'] = features['x'][0, 32, :, :, 0]

    expected_output_size = [1, 128, 128, 1]  # [B, W, H, C]
    [tf.summary.image(name, tf.reshape(image, expected_output_size))
     for name, image in my_image_summaries.items()]

    # Track RMSE
    acc             = tf.metrics.accuracy
    prec            = tf.metrics.precision
    auc             = tf.metrics.auc
    eval_metric_ops = {"accuracy":  acc(labels['y'],  clfnet_output_ops['y_']),
                       "precision": prec(labels['y'], clfnet_output_ops['y_']),
                       "auc":       prec(labels['y'], clfnet_output_ops['y_'])}

    # Return EstimatorSpec Object
    return tf.estimator.EstimatorSpec(mode            = mode,
                                      predictions     = clfnet_output_ops,
                                      loss            = loss,
                                      train_op        = train_op,
                                      eval_metric_ops = eval_metric_ops)

# Training Script

In [5]:
count_steps = []
count_loss  = []


EVAL_EVERY_N_STEPS = 500
EVAL_STEPS         = 10

PATCH              = 96
NUM_CLASSES        = 2
NUM_CHANNELS       = 63
CONNECT            = 'max'

BATCH_SIZE         = 3
SHUFFLE_CACHE_SIZE = 64

MAX_STEPS          = 150000

In [6]:
# Read Arguments
with open('config.json') as f:
    run_config = json.load(f)
    
train_filenames = pd.read_csv(
    '.\csvIII\Lung_CV-Training-Fold-1.csv', dtype=object, keep_default_na=False,
    na_values=[]).values

val_filenames = pd.read_csv(
    '.\csvIII\Lung_CV-Validation-Fold-1.csv', dtype=object, keep_default_na=False,
    na_values=[]).values

In [7]:
# Set DLTK Reader Parameters (No. of Patches, Patch Size) 
reader_params = {'n_patches': 2,
                 'patch_size': [PATCH, PATCH, PATCH],   # Target Patch Size
                 'extract_patches': True,                # Enable Training Mode Patch Extraction
                 'augmentation':True}
# Set Patch Dimensions
reader_patch_shapes = {'features': {'x': reader_params['patch_size'] + [NUM_CHANNELS]},
                       'labels':   {'y': [1]}}
# Initiate Data Reader + Patch Extraction
reader = Reader(read_fn,
              {'features': {'x': tf.float32},
               'labels':   {'y': tf.int32}})

In [8]:
# Create Input Functions + Queue Initialisation Hooks for Training/Validation Data
train_input_fn, train_qinit_hook = reader.get_inputs(
    file_references       = train_filenames,
    mode                  = tf.estimator.ModeKeys.TRAIN,
    example_shapes        = reader_patch_shapes,
    batch_size            = BATCH_SIZE,
    shuffle_cache_size    = SHUFFLE_CACHE_SIZE,
    params                = reader_params)
val_input_fn, val_qinit_hook = reader.get_inputs(
    file_references       = val_filenames,
    mode                  = tf.estimator.ModeKeys.EVAL,
    example_shapes        = reader_patch_shapes,
    batch_size            = BATCH_SIZE,
    shuffle_cache_size    = SHUFFLE_CACHE_SIZE,
    params                = reader_params)

In [9]:
# Instantiate Neural Network Estimator
nn = tf.estimator.Estimator(
    model_fn             = model_fn,
    model_dir            = './weights/17072019/Fold_1',
    params               = run_config,
    config               = tf.estimator.RunConfig())

# Hooks for Validation Summaries
val_summary_hook = tf.contrib.training.SummaryAtEndHook(
    os.path.join('./weights/17072019/Fold_1', 'eval'))
step_cnt_hook = tf.train.StepCounterHook(every_n_steps=EVAL_EVERY_N_STEPS,
                                         output_dir='./weights/17072019/Fold_1')

INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_device_fn': None, '_experimental_distribute': None, '_task_id': 0, '_eval_distribute': None, '_evaluation_master': '', '_save_checkpoints_secs': 600, '_task_type': 'worker', '_protocol': None, '_model_dir': './weights/17072019/Fold_1', '_is_chief': True, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_train_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002289B119CC0>, '_num_ps_replicas': 0, '_log_step_count_steps': 100, '_master': '', '_service': None, '_save_summary_steps': 100, '_global_id_in_cluster': 0}


In [10]:
input = tf.zeros([3, 96, 96, 96, 63], dtype=tf.float32)
labels = [1]
input.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'zeros:0' shape=(3, 96, 96, 96, 63) dtype=float32>>

In [ ]:
# Segmentation Model Definition (3D Residual U-Net)
segnet_output_ops = residual_unet_3d(
    inputs              = input,                               # Input: Patches (dimensions=128cube; defined by 'patch_size')
    num_classes         = 2,
    num_res_units       = 3,
    filters             = [16, 64, 128, 256, 512],
    strides             = ((1, 1, 1), (2, 2, 2), (2, 2, 2), (2, 2, 2), (1, 1, 1)),
    mode                = tf.estimator.ModeKeys.EVAL,
    activation          = lrelu,
    kernel_initializer  = tf.initializers.variance_scaling(distribution='uniform'),
    bias_initializer    = tf.zeros_initializer(),
    kernel_regularizer  = None,
    bias_regularizer    = None)

In [ ]:
segnet_output_ops

In [11]:
# Classification Model Definition (3D ResNet)
clfnet_output_ops = resnet_3d(
    inputs              = input,                # Input: Output of 3D Residual U-Net (dimensions=128cube)
    num_res_units       = 2,
    num_classes         = NUM_CLASSES,
    filters             = (16, 32, 64, 128, 256),
    strides             = ((1, 1, 1), (2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2)),
    kernel_initializer  = tf.initializers.variance_scaling(distribution='uniform'),
    bias_initializer    = tf.zeros_initializer(),
    mode                = tf.estimator.ModeKeys.TRAIN,
    kernel_regularizer  = tf.contrib.layers.l2_regularizer(1e-3))

Instructions for updating:
Use keras.layers.conv3d instead.
Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Init conv tensor shape (3, 96, 96, 96, 16)
Instructions for updating:
Use keras.layers.max_pooling3d instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
INFO:tensorflow:Encoder at res_scale 1 tensor shape: (3, 48, 48, 48, 32)
INFO:tensorflow:Encoder at res_scale 2 tensor shape: (3, 24, 24, 24, 64)
INFO:tensorflow:Encoder at res_scale 3 tensor shape: (3, 12, 12, 12, 128)
INFO:tensorflow:Encoder at res_scale 4 tensor shape: (3, 6, 6, 6, 256)
INFO:tensorflow:Global pool shape (3, 256)
Instructions for updating:
Use keras.layers.dense instead.
INFO:tensorflow:Output tensor shape (3, 2)


In [ ]:
xclfnet_output_ops = resnet_3d(
    inputs              = tf.concat(values=[input,segnet_output_ops['logits']], axis=4),                # Input: Output of 3D Residual U-Net (dimensions=128cube)
    num_res_units       = 2,
    num_classes         = NUM_CLASSES,
    filters             = (16, 32, 64, 128, 256),
    strides             = ((1, 1, 1), (2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2)),
    kernel_initializer  = tf.initializers.variance_scaling(distribution='uniform'),
    bias_initializer    = tf.zeros_initializer(),
    mode                = tf.estimator.ModeKeys.TRAIN,
    kernel_regularizer  = tf.contrib.layers.l2_regularizer(1e-3))

In [ ]:
# Loss Function
one_hot_labels = tf.reshape(tf.one_hot(labels, depth=NUM_CLASSES), [-1, NUM_CLASSES])
loss = tf.losses.softmax_cross_entropy(
    onehot_labels      = one_hot_labels,
    logits             = xclfnet_output_ops['logits'])

## Tensors

In [ ]:
patch =   tf.zeros([1, 128, 128, 128, 1], dtype=tf.float32)
feature = tf.ones([1, 128, 128, 128, 2], dtype=tf.float32)
patch.get_shape

In [ ]:
tf.concat(values=[patch,feature], axis=4)

In [ ]:
m = np.array([0,1,6])
m.sum()

# Deploy Script

In [ ]:
READER_PARAMS = {'extract_patches':          False,
                 'augmentation':             False,
                 'n_patches':                    1,
                 'patch_size':      [128, 128, 128]}

model_path = 'idealU'

In [ ]:
id_list              =  []     # Patient ID
probability_list     =  []     # Probabilities
label_list           =  []     # Labels
class_1_list         =  []     # Class 1
class_2_list         =  []     # Class 2

# Read CSV with Validation/Test Set
file_names = pd.read_csv(
    '.\csv\Lung_CV-Training-Fold-1.csv',
    dtype=object,
    keep_default_na=False,
    na_values=[]).values


# Load Trained Model
export_dir = \
    [os.path.join(model_path, o) for o in sorted(os.listdir(model_path))
     if os.path.isdir(os.path.join(model_path, o)) and o.isdigit()][-1]
print('Loading from {}'.format(export_dir))

my_predictor = predictor.from_saved_model(export_dir)
# Iterate through Files, Predict on the Full Volumes, Compute Dice
accuracy = []
for output in read_fn(file_references = file_names,
                      mode            = tf.estimator.ModeKeys.PREDICT,
                      params          = READER_PARAMS):
    
    t0 = time.time()  # Timing Function
    # Parse Data Reader Output
    img      = output['features']['x']
    lbl      = output['labels']['y']
    test_id  = output['img_id']
    # Decompose Volumes into Patches        
    num_crop_predictions = 8
    crop_batch = extract_random_example_array(
        image_list     = img,
        example_size   = [128, 128, 128],
        n_examples     = num_crop_predictions)


In [ ]:
my_predictor._fetch_tensors['y_prob']
# my_predictor._feed_tensors['x']: crop_batch})

In [ ]:
my_predictor._fetch_tensors

In [ ]:
my_predictor

In [ ]:
y_probs = my_predictor._fetch_tensors['y_prob']
y_probsx = [(my_predictor._fetch_tensors['y_prob'])]
y_probsx

# residual_3DP

In [ ]:
from __future__ import unicode_literals
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

import tensorflow as tf

from dltk.core.residual_unit import vanilla_residual_unit_3d
from dltk.core.upsample import linear_upsample_3d
from dltk.core.activations import leaky_relu



# Auxiliary Functions
def upsample_and_concat(inputs, inputs2, strides=(2, 2, 2)):
    """Upsampling and concatenation layer according to [1].

    [1] O. Ronneberger et al. U-Net: Convolutional Networks for Biomedical Image
        Segmentation. MICCAI 2015.

    Args:
        inputs (TYPE): Input features to be upsampled.
        inputs2 (TYPE): Higher resolution features from the encoder to
            concatenate.
        strides (tuple, optional): Upsampling factor for a strided transpose
            convolution.

    Returns:
        tf.Tensor: Upsampled feature tensor
    """
    assert len(inputs.get_shape().as_list()) == 5, \
        'inputs are required to have a rank of 5.'
    assert len(inputs.get_shape().as_list()) == len(inputs2.get_shape().as_list()), \
        'Ranks of input and input2 differ'

    # Upsample inputs
    inputs = linear_upsample_3d(inputs, strides)

    return tf.concat(axis=-1, values=[inputs2, inputs])


# Target Model
def residual_3DPNet (inputs,
                     num_classes,
                     mode                        = tf.estimator.ModeKeys.EVAL,
                     
                     seg__num_res_units          = 1,
                     seg__filters                = (16, 32, 64, 128),
                     seg__strides                = ((1, 1, 1), (2, 2, 2), (2, 2, 2), (2, 2, 2)),
                     seg__use_bias               = False,
                     seg__activation             = leaky_relu,
                     seg__kernel_initializer     = tf.initializers.variance_scaling(distribution='uniform'),
                     seg__bias_initializer       = tf.zeros_initializer(),
                     seg__kernel_regularizer     = None,
                     seg__bias_regularizer       = None,
                     seg__bottleneck             = False,
                     
                     clf__num_res_units          = 1,
                     clf__filters                = (16, 32, 64, 128),
                     clf__strides                = ((1, 1, 1), (2, 2, 2), (2, 2, 2), (2, 2, 2)), 
                     clf__use_bias               = False,
                     clf__activation             = tf.nn.relu6,
                     clf__kernel_initializer     = tf.initializers.variance_scaling(distribution='uniform'),
                     clf__bias_initializer       = tf.zeros_initializer(),
                     clf__kernel_regularizer     = None, 
                     clf__bias_regularizer       = None):

    
    # RESIDUAL 3D U-NET (residual_3D_unet) // SEGMENTATION {
    """
    Image segmentation network based on a flexible UNET architecture [1]
    using residual units [2] as feature extractors. Downsampling and
    upsampling of features is done via strided convolutions and transpose
    convolutions, respectively. On each resolution scale s are
    num_residual_units with filter size = filters[s]. strides[s] determine
    the downsampling factor at each resolution scale.
     
    [1] O. Ronneberger et al. U-Net: Convolutional Networks for Biomedical Image
        Segmentation. MICCAI 2015.
    [2] K. He et al. Identity Mappings in Deep Residual Networks. ECCV 2016.
    
    Args:
        inputs (tf.Tensor): Input feature tensor to the network (rank 5
            required).
        num_classes (int): Number of output classes.
        num_res_units (int, optional): Number of residual units at each
            resolution scale.
        filters (tuple, optional): Number of filters for all residual units at
            each resolution scale.
        strides (tuple, optional): Stride of the first unit on a resolution
            scale.
        mode (TYPE, optional): One of the tf.estimator.ModeKeys strings: TRAIN,
            EVAL or PREDICT
        use_bias (bool, optional): Boolean, whether the layer uses a bias.
        activation (optional): A function to use as activation function.
        kernel_initializer (TYPE, optional): An initializer for the convolution
            kernel.
        bias_initializer (TYPE, optional): An initializer for the bias vector.
            If None, no bias will be applied.
        kernel_regularizer (None, optional): Optional regularizer for the
            convolution kernel.
        bias_regularizer (None, optional): Optional regularizer for the bias
            vector.
    
    Returns:
        dict: dictionary of output tensors
    
    }"""
    # Input:  CT Patch Features 
    # Output: Segmentation Feature Maps
    
    
    # INITIALIZATION
    assert len(seg__strides) == len(seg__filters)
    assert len(inputs.get_shape().as_list()) == 5, \
        'Inputs are required to have a rank of 5.'
    
    seg__conv_params = {'padding':              'same',
                        'use_bias':             seg__use_bias,
                        'kernel_initializer':   seg__kernel_initializer,
                        'bias_initializer':     seg__bias_initializer,
                        'kernel_regularizer':   seg__kernel_regularizer,
                        'bias_regularizer':     seg__bias_regularizer}
    x = inputs
    
    # Initial Convolution with (seg__filters[0])
    x = tf.layers.conv3d(inputs       = x,
                         filters      = seg__filters[0],
                         kernel_size  = (3, 3, 3),
                         strides      = seg__strides[0],
                         **seg__conv_params)
    tf.logging.info('Segmentation Network: Initial Conv3D Tensor Shape: {}'.format(x.get_shape()))
    
    
    
    
    # FEATURE EXTRACTOR // ENCODER:
    # Residual Blocks with (seg__num_res_units) at Different Resolution Scales (res_scales)
    res_scales = [x]
    saved_strides = []
    for res_scale in range(1, len(seg__filters)):
     
        # Features are Downsampled via Strided Convolutions ('seg__strides')
        with tf.variable_scope('enc_unit_{}_0'.format(res_scale)):
            x = vanilla_residual_unit_3d(
                inputs            = x,
                out_filters       = seg__filters[res_scale],
                strides           = seg__strides[res_scale],
                activation        = seg__activation,
                mode              = mode)
        saved_strides.append(seg__strides[res_scale])
        
        for i in range(1, seg__num_res_units):
            with tf.variable_scope('enc_unit_{}_{}'.format(res_scale, i)):
                x = vanilla_residual_unit_3d(
                    inputs        = x,
                    out_filters   = seg__filters[res_scale],
                    strides       = (1, 1, 1),
                    activation    = seg__activation,
                    mode          = mode)
        res_scales.append(x)
        tf.logging.info('Segmentation Network: Encoder at "res_scale" {} Tensor Shape: {}'.format(
            res_scale, x.get_shape()))
    
    
    
    
    # RESTORE SPATIAL DIMENSION // DECODER:
    # Upsample and Concatenate Layers and Reconstruct Predictions to Higher Resolution Scales
    for res_scale in range(len(seg__filters) - 2, -1, -1):
        
        with tf.variable_scope('up_concat_{}'.format(res_scale)):
            x = upsample_and_concat(
                inputs           = x,
                inputs2          = res_scales[res_scale],
                strides          = saved_strides[res_scale])
        
        for i in range(0, seg__num_res_units):
            with tf.variable_scope('dec_unit_{}_{}'.format(res_scale, i)):
                x = vanilla_residual_unit_3d(
                    inputs       = x,
                    out_filters  = seg__filters[res_scale],
                    strides      = (1, 1, 1),
                    mode         = mode)
        tf.logging.info('Segmentation Network: Decoder at "res_scale" {} Tensor Shape: {}'.format(
            res_scale, x.get_shape()))
    
    
    
    
    # BOTTLENECK CONVOLUTION
    if seg__bottleneck:
     with tf.variable_scope('last'):
        
         x = tf.layers.conv3d(inputs         = x,
                              filters        = num_classes,
                              kernel_size    = (1, 1, 1),
                              strides        = (1, 1, 1),
                              **seg__conv_params)
     tf.logging.info('Segmentation Network: Bottleneck Output Tensor Shape: {}'.format(x.get_shape()))
    
    
    
    
    # OUTPUT
    residual3Dunet_output = x
    
    
    
    
    
    
    
    
    
    
    
    
    
    # 3D RESNET (resnet_3d) // CLASSIFICATION {
    """
    Regression/classification network based on a flexible resnet
    architecture [1] using residual units proposed in [2]. The downsampling
    of features is done via strided convolutions. On each resolution scale s
    are num_convolutions with filter size = filters[s]. strides[s]
    determine the downsampling factor at each resolution scale.
    
    [1] K. He et al. Deep residual learning for image recognition. CVPR 2016.
    [2] K. He et al. Identity Mappings in Deep Residual Networks. ECCV 2016.
    
    Args:
        inputs (tf.Tensor): Input feature tensor to the network (rank 5
            required).
        num_classes (int): Number of output channels or classes.
        num_res_units (int, optional): Number of residual units per resolution
            scale.
        filters (tuple, optional): Number of filters for all residual units at
            each resolution scale.
        strides (tuple, optional): Stride of the first unit on a resolution
            scale.
        mode (TYPE, optional): One of the tf.estimator.ModeKeys strings: TRAIN,
            EVAL or PREDICT
        use_bias (bool, optional): Boolean, whether the layer uses a bias.
        activation (optional): A function to use as activation function.
        kernel_initializer (TYPE, optional): An initializer for the convolution
            kernel.
        bias_initializer (TYPE, optional): An initializer for the bias vector.
            If None, no bias will be applied.
        kernel_regularizer (None, optional): Optional regularizer for the
            convolution kernel.
        bias_regularizer (None, optional): Optional regularizer for the bias
            vector.
    
    Returns:
        dict: dictionary of output tensors
    
    }"""
    # Input:  CT Patch Features (concat) Segmentation Feature Maps
    # Output: Classification Scores (logits, y_prob, y_)


    # INITIALIZATION
    resnet3D_output = {}
    assert len(clf__strides) == len(clf__filters)
    assert len(residual3Dunet_output.get_shape().as_list()) == 5, \
        'Inputs are required to have a rank of 5.'
    
    relu_op = tf.nn.relu6
    
    clf__conv_params = {'padding':              'same',
                        'use_bias':             clf__use_bias,
                        'kernel_initializer':   clf__kernel_initializer,
                        'bias_initializer':     clf__bias_initializer,
                        'kernel_regularizer':   clf__kernel_regularizer,
                        'bias_regularizer':     clf__bias_regularizer}
    
    # Concatenated CT and Segmentation Features --> Input Feed 
    x = tf.concat(values=[inputs, residual3Dunet_output], axis=4)
    
    # Initial Convolution with (clf__filters[0])
    k = [s * 2 if s > 1 else 3 for s in clf__strides[0]]
    x = tf.layers.conv3d(x, clf__filters[0], k, clf__strides[0], **clf__conv_params)
    tf.logging.info('Classification Network: Initial Conv3D Tensor Shape: {}'.format(x.get_shape()))
    
    
       
    
    # FEATURE EXTRACTOR // ENCODER:
    # Residual Blocks with (clf__num_res_units) at Different Resolution Scales (res_scales)
    res_scales = [x]
    saved_strides = []
    for res_scale in range(1, len(clf__filters)):
        
        # Features are Downsampled via Strided Convolutions ('clf__strides')
        with tf.variable_scope('unit_{}_0'.format(res_scale)):
            x = vanilla_residual_unit_3d(
                inputs                = x,
                out_filters           = clf__filters[res_scale],
                strides               = clf__strides[res_scale],
                activation            = clf__activation,
                mode                  = mode)
        saved_strides.append(clf__strides[res_scale])
        
        for i in range(1, clf__num_res_units):
            with tf.variable_scope('unit_{}_{}'.format(res_scale, i)):
                x = vanilla_residual_unit_3d(
                    inputs            = x,
                    out_filters       = clf__filters[res_scale],
                    strides           = (1, 1, 1),
                    activation        = clf__activation,
                    mode              = mode)
        res_scales.append(x)
        tf.logging.info('Classification Network: Encoder at "res_scale" {} Tensor Shape: {}'.format(
            res_scale, x.get_shape()))




    
    # GLOBAL POOLING + FINAL LAYER
    with tf.variable_scope('pool'):
        x = tf.layers.batch_normalization(
            x, training=mode == tf.estimator.ModeKeys.TRAIN)
        x = relu_op(x)
        
        axis = tuple(range(len(x.get_shape().as_list())))[1:-1]
        x = tf.reduce_mean(x, axis=axis, name='global_avg_pool')
        
        tf.logging.info('Classification Network: Global Pooling Tensor Shape: {}'.format(x.get_shape()))
    
    with tf.variable_scope('last'):
        x = tf.layers.dense(inputs                = x,
                            units                 = num_classes,
                            activation            = None,
                            use_bias              = clf__conv_params['use_bias'],
                            kernel_initializer    = clf__conv_params['kernel_initializer'],
                            bias_initializer      = clf__conv_params['bias_initializer'],
                            kernel_regularizer    = clf__conv_params['kernel_regularizer'],
                            bias_regularizer      = clf__conv_params['bias_regularizer'],
                            name                  = 'hidden_units')
        
        tf.logging.info('Classification Network: Output Tensor Shape: {}'.format(x.get_shape()))



    
    # OUTPUT
    resnet3D_output['logits'] = x
    
    with tf.variable_scope('pred'):
        
        y_prob = tf.nn.softmax(x)
        resnet3D_output['y_prob'] = y_prob
        
        y_ = tf.argmax(x, axis=-1) \
            if num_classes > 1 \
            else tf.cast(tf.greater_equal(x[..., 0], 0.5), tf.int32)
        resnet3D_output['y_'] = y_
    
    return resnet3D_output

In [ ]:
# Model Definition (residual_3DPNet)
model_output_ops = residual_3DPNet(
    # Input: Patches (dimensions=128cube,channel=1; defined by 'patch_size')
    inputs                   = input,                               
    num_classes              = 2,
    mode                     = tf.estimator.ModeKeys.TRAIN,
    
    seg__num_res_units       = 1,
    seg__filters             = [8, 16],
    seg__strides             = ((1, 1, 1), (1, 1, 1)),
    seg__activation          = lrelu,
    seg__kernel_initializer  = tf.initializers.variance_scaling(distribution='uniform'),
    seg__bias_initializer    = tf.zeros_initializer(),
    seg__kernel_regularizer  = None,
    seg__bottleneck          = False,
            
    clf__num_res_units       = 2,
    clf__filters             = (16, 32, 64, 128, 256),
    clf__strides             = ((1, 1, 1), (2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2)),
    clf__kernel_initializer  = tf.initializers.variance_scaling(distribution='uniform'),
    clf__bias_initializer    = tf.zeros_initializer(),
    clf__kernel_regularizer  = tf.contrib.layers.l2_regularizer(1e-3))

In [ ]:
x = tf.concat(values=[input, input], axis=4)
x

In [ ]:
input.dtype

## Visualize Feature Map

In [ ]:
input = tf.zeros([1, 128, 128, 128, 4], dtype=tf.float32)
F     = tf.concat(values=[input, input], axis=4)
shape = F.get_shape().as_list()
ydim        = shape[2]
xdim        = shape[3]
featuremaps = shape[4]

In [ ]:
F = tf.slice(F,(0,0,0,0,0),(1,1,-1,-1,-1)) 
F = tf.reshape(F,(ydim,xdim,featuremaps))
F.get_shape()

In [ ]:
ydim += 4
xdim += 4
F = tf.image.resize_image_with_crop_or_pad(F,ydim,xdim)

In [ ]:
F = tf.reshape(F,(ydim,xdim,4,2)) 
F = tf.transpose(F,(2,0,3,1))
F = tf.reshape(F,(1,4*ydim,2*xdim,1))

In [ ]:
tf.summary.image('Segmentation Feature Maps', F, 20)